In [13]:

import os
import json
import posixpath # Forces Unix-style paths
from sklearn.model_selection import train_test_split

# --- LOCAL PATHS (Where files are now) ---
LOCAL_IMG_DIR = r'Z:\welsh_dragon_20251208\drr\drr_images'
LOCAL_MASK_DIR = r'Z:\welsh_dragon_20251208\mask'
OUTPUT_DIR = './welsh_jsons'

# --- REMOTE PATHS (How they should look in JSON) ---
REMOTE_BASE = "/home/vy"
# Subfolder names inside /home/vy/
REMOTE_IMG_SUB = "welsh_dragon_20251208/drr/drr_images" 
REMOTE_MASK_SUB = "welsh_dragon_20251208/mask"

TRAIN_RATIO, VAL_RATIO, TEST_RATIO = 0.8, 0.1, 0.1

def generate_unix_jsons():
    img_files = sorted([f for f in os.listdir(LOCAL_IMG_DIR) if not f.startswith('.')])
    mask_files = sorted([f for f in os.listdir(LOCAL_MASK_DIR) if not f.startswith('.')])
    
    num_samples = min(len(img_files), len(mask_files))
    data_pairs = []

    for i in range(num_samples):
        # Construct Unix paths using posixpath
        remote_img = posixpath.join(REMOTE_BASE, REMOTE_IMG_SUB, img_files[i])
        remote_mask = posixpath.join(REMOTE_BASE, REMOTE_MASK_SUB, mask_files[i])
        
        data_pairs.append({
            "drr": remote_img,
            "mask": remote_mask
        })

    # Split
    train_data, temp = train_test_split(data_pairs, train_size=TRAIN_RATIO, random_state=42)
    val_data, test_data = train_test_split(temp, train_size=0.5, random_state=42)

    os.makedirs(OUTPUT_DIR, exist_ok=True)
    for name, data in [("train.json", train_data), ("val.json", val_data), ("test.json", test_data)]:
        with open(os.path.join(OUTPUT_DIR, name), 'w') as f:
            json.dump(data, f, indent=4)
        print(f"Created {name} with Unix paths.")

if __name__ == "__main__":
    generate_unix_jsons()

Created train.json with Unix paths.
Created val.json with Unix paths.
Created test.json with Unix paths.


In [14]:
import os
import json

# --- CONFIGURATION ---
JSON_FILES = ['./welsh_jsons/train.json', './welsh_jsons/val.json', './welsh_jsons/test.json']
NEW_FOLDER_DIR = 'Z:\\welsh_dragon_20251208\\xray'
NEW_KEY_NAME = 'xray'  # The key that will appear in your JSON
ORIGINAL_IMG_DIR = 'Z:\\welsh_dragon_20251208\\drr\\drr_images' # Needed to find the original index
import os
import json
import posixpath

# --- CONFIGURATION ---
JSON_FILES = ['./welsh_jsons/train.json', './welsh_jsons/val.json', './welsh_jsons/test.json']
LOCAL_IMG_DIR = r'Z:\welsh_dragon_20251208\drr\drr_images' # Used to find the original index
LOCAL_NEW_DIR = r'Z:\welsh_dragon_20251208\xray'

REMOTE_BASE = "/home/vy"
REMOTE_NEW_SUB = "welsh_dragon_20251208/xray" # Folder name on Linux
NEW_KEY = "xray"  # The key that will appear in your JSON

def update_jsons_unix():
    # 1. Map filenames to their index based on original local sorting
    all_imgs = sorted([f for f in os.listdir(LOCAL_IMG_DIR) if not f.startswith('.')])
    all_new = sorted([f for f in os.listdir(LOCAL_NEW_DIR) if not f.startswith('.')])

    # index_map: { "filename.jpg": "/home/vy/extra_data/new_file.png" }
    index_map = {}
    for i in range(min(len(all_imgs), len(all_new))):
        unix_path = posixpath.join(REMOTE_BASE, REMOTE_NEW_SUB, all_new[i])
        index_map[all_imgs[i]] = unix_path

    # 2. Update JSON files
    for json_path in JSON_FILES:
        if not os.path.exists(json_path): continue

        with open(json_path, 'r') as f:
            data = json.load(f)

        for entry in data:
            # Get filename from the existing Unix path in JSON
            # posixpath.basename works correctly on forward-slash paths
            filename = posixpath.basename(entry['drr'])
            
            if filename in index_map:
                entry[NEW_KEY] = index_map[filename]

        with open(json_path, 'w') as f:
            json.dump(data, f, indent=4)
        print(f"Updated {json_path} with Unix paths for '{NEW_KEY}'.")

if __name__ == "__main__":
    update_jsons_unix()

Updated ./welsh_jsons/train.json with Unix paths for 'xray'.
Updated ./welsh_jsons/val.json with Unix paths for 'xray'.
Updated ./welsh_jsons/test.json with Unix paths for 'xray'.
